In [2]:
# import os
# 
# from pandas import read_csv
# from torch import save
# from torch.optim import AdamW
# from sklearn.model_selection import train_test_split
# 
# from transformers import AutoTokenizer, AutoModel
# 
# from src.dataset import load_dataset, Species, Modification, SeqBunch
# from src.utils.transformers import encode_seq_bunch, make_dataloader, train_epoch, calculate_acc_dataset

import transformers

In [8]:
config = transformers.BertConfig.from_pretrained('zhihan1996/DNABERT-2-117M', num_labels=3)
model = transformers.AutoModelForSequenceClassification.from_config(config)
model
# model = transformers.AutoModelForSequenceClassification.from_pretrained(
#     'zhihan1996/DNABERT-2-117M',
#     cache_dir=training_args.cache_dir,
#     num_labels=train_dataset.num_labels,
#     trust_remote_code=True,
#     config=config,
# )

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(4096, 768, padding_idx=0)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertUnpadAttention(
            (self): BertUnpadSelfAttention(
              (dropout): Dropout(p=0.0, inplace=False)
              (Wqkv): Linear(in_features=768, out_features=2304, bias=True)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
          )
          (mlp): BertGatedLinearUnitMLP(
            (gated_layers): Linear(in_feat

In [5]:
DEVICE = 'mps'
MODEL = 'zhihan1996/DNABERT-2-117M'

EXPERIMENT_NAME = 'dna-bert-2'

In [6]:
import torch
from transformers import AutoTokenizer, AutoModel, BertConfig

tokenizer = AutoTokenizer.from_pretrained("zhihan1996/DNABERT-2-117M", trust_remote_code=True)
config = BertConfig.from_pretrained("zhihan1996/DNABERT-2-117M")
model = AutoModel.from_config(config)

In [7]:
dataset = load_dataset(Species.human, Modification.psi)

# raw_data = read_csv('/Users/arish/Workspace/experiments/rna_modification/notebook/data/sub_sampled.csv', header=None)
# 
# dataset = SeqBunch(
#     samples=raw_data.drop(1, axis=1).rename({0: 'sequence'}, axis=1),
#     targets=raw_data[1]
# )

In [8]:
dna = "ACGTAGCATCGGATCTATCTATCGACACTTGGTTATCGATCTACGAGCATCTCGTTAGC"
inputs = tokenizer(dna, return_tensors='pt')["input_ids"]
hidden_states = model(inputs)[0]  # [1, sequence_length, 768]

# # embedding with mean pooling
# embedding_mean = torch.mean(hidden_states[0], dim=0)
# print(embedding_mean.shape)  # expect to be 768

# embedding with max pooling
embedding_max = torch.max(hidden_states[0], dim=0)[0]
print(embedding_max)  # expect to be 768

tensor([ 6.6045e-01,  1.8410e+00,  1.6546e+00,  8.0555e-02,  1.2770e+00,
         1.7988e+00,  9.3307e-01, -3.8802e-01, -4.4682e-01,  2.2355e+00,
         1.8718e+00,  1.3079e+00,  1.9879e+00,  5.1020e-01,  2.2848e+00,
         2.8768e+00,  1.1841e+00,  8.6278e-01,  1.2456e+00,  2.3437e+00,
         1.5788e+00,  1.2688e+00,  2.0612e+00,  1.8073e+00,  1.3071e-01,
         1.6686e+00,  6.6358e-01,  1.9507e+00,  2.4020e+00,  1.4135e+00,
         2.5022e+00, -5.8453e-02,  9.4066e-01,  7.4905e-01,  1.2645e+00,
         2.6458e-01,  2.6435e+00,  1.4058e+00,  1.1627e+00,  1.9699e+00,
         2.1832e+00,  1.1788e+00,  2.1455e+00,  1.0564e-01,  1.4664e+00,
         3.1770e+00,  4.1230e-02,  2.0617e+00,  1.3036e+00,  1.3882e+00,
         8.9200e-01,  1.3037e+00,  1.9561e-01,  4.6081e-01,  1.4482e+00,
        -7.6959e-02,  2.1246e+00,  5.3354e-01,  7.3946e-01,  2.1330e+00,
         5.6788e-01, -5.7495e-01,  1.1488e+00,  1.1826e+00,  1.4375e+00,
         2.5357e-01,  2.5861e+00,  1.5943e+00,  1.0

In [13]:
inputs

{'input_ids': tensor([[   1,    5,  194,   32,  757, 1239, 2092,  294,   24,  359,   88,   93,
           32,   75,   77,   19,    2]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [5]:
dataset.samples

,sequence
0,AAAUAUGUCUGAUAAAAGAGUUACUUUGAUAGAGUAAAUAA
1,AAUAUGUCUGAUAAAAGAGUUACUUUGAUAGAGUAAAUAAU
2,AAGGUCAGCUAAAUAAGCUAUCGGGCCCAUACCCCGAAAAU
3,GGGCCCAUACCCCGAAAAUGUUGGUUAUAUCCUUCCCGUAC
4,AUAUUAGAAAAACCAUUUCAUAACUUUGUCAAAGUUAAAUU
...,...
7909,CAUCACGCUAUCCUUUGUUGUCAUGCUUCGUGCCUCGGCGG
7910,CGGGCGUGACUAGAUGUCUCUCUGGCUUCCUCCAAUAUAUU
7911,CCACGCUACCAUUACGGCUAUGAGGGUCUAGACUUACGAUA
7912,UGCCACAUGCAGUGGAGGCGUUGACCCGAGAUGUUGUUCGA


In [5]:
sequences, labels = encode_seq_bunch(dataset, tokenizer, True)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [6]:
x_train, x_test, y_train, y_test = train_test_split(sequences, labels, test_size=0.2)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2)

In [7]:
train_dataloader = make_dataloader(x_train, y_train)
test_dataloader = make_dataloader(x_test, y_test)
val_dataloader = make_dataloader(x_val, y_val)

In [8]:
model = BertForSequenceClassification.from_pretrained(MODEL, num_labels=2)
model.to(DEVICE)

None

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
optimizer = AdamW(model.parameters(), lr=1e-5)

In [10]:
old_val_acc = 0
old_train_acc = 0
old_model_name = ''

In [11]:
TOTAL_EPOCHS = 0

In [15]:
for epoch in range(1, 5 + 1):
    TOTAL_EPOCHS += 1

    train_acc, val_acc = train_epoch(TOTAL_EPOCHS, DEVICE, model, optimizer, train_dataloader, val_dataloader)
    if train_acc > old_train_acc and val_acc > old_val_acc:
        if old_model_name != '':
            os.unlink(old_model_name)
        old_val_acc = val_acc
        old_train_acc = train_acc
        old_model_name = f'{EXPERIMENT_NAME}_ep-{TOTAL_EPOCHS}_tacc-{train_acc:.2}_vacc-{val_acc:.2}.pt'
        save(model, old_model_name)

  0%|          | 0/253 [00:00<?, ?it/s]

  0%|          | 0/253 [00:00<?, ?it/s]

  0%|          | 0/253 [00:00<?, ?it/s]

  0%|          | 0/253 [00:00<?, ?it/s]

  0%|          | 0/253 [00:00<?, ?it/s]

In [13]:
calculate_acc_dataset(DEVICE, model, test_dataloader)

0.7553528945281522